<a href="https://colab.research.google.com/github/masaki0914/data_analysis_100knocks/blob/main/%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90100%E6%9C%AC%E3%83%8E%E3%83%83%E3%82%AF_%E7%AC%AC5%E7%AB%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5章 顧客の退会を予測する

## データを読み込んで利用データを整形

In [8]:
# 警告(worning)の非表示化
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
customer = pd.read_csv("customer_join.csv")
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [5]:
uselog_months = pd.read_csv("use_log_months.csv")
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [6]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


##退会前月の退会顧客データを作成

In [10]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
# ループの代わりにapplyメソッドを使用して、exit_date列を直接日付時刻型として作成します。
exit_customer["exit_date"] = exit_customer["end_date"].apply(lambda x: x - relativedelta(months=1))
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [11]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


##継続顧客のデータを作成

In [12]:
from typing_extensions import Container
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [13]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201809,OA909231,9,8.0,XX,C03,M,2018-01-01,NaN,CA1,...,ナイト,6000.0,通常,7.083333,7.0,12.0,3.0,1.0,2019-04-30,15.0
1,201805,HD862235,4,4.0,XXXXX,C01,M,2015-08-01,NaN,CA1,...,オールタイム,10500.0,通常,4.166667,4.0,6.0,1.0,1.0,2019-04-30,44.0
2,201811,IK192713,5,6.0,XXXX,C01,M,2016-04-01,NaN,CA1,...,オールタイム,10500.0,通常,3.916667,3.5,6.0,2.0,1.0,2019-04-30,36.0
3,201810,IK429268,6,7.0,XXXX,C02,F,2018-04-06,NaN,CA3,...,デイタイム,7500.0,入会費無料,7.500000,7.0,10.0,6.0,1.0,2019-04-30,12.0
4,201903,AS460629,8,6.0,XXXXX,C01,M,2018-07-14,NaN,CA1,...,オールタイム,10500.0,通常,7.555556,8.0,9.0,6.0,1.0,2019-04-30,9.0


In [14]:
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201809,OA909231,9,8.0,XX,C03,M,2018-01-01,NaT,CA1,...,6000.0,通常,7.083333,7.0,12.0,3.0,1.0,2019-04-30,15.0,NaT
1,201805,HD862235,4,4.0,XXXXX,C01,M,2015-08-01,NaT,CA1,...,10500.0,通常,4.166667,4.0,6.0,1.0,1.0,2019-04-30,44.0,NaT
2,201811,IK192713,5,6.0,XXXX,C01,M,2016-04-01,NaT,CA1,...,10500.0,通常,3.916667,3.5,6.0,2.0,1.0,2019-04-30,36.0,NaT
3,201810,IK429268,6,7.0,XXXX,C02,F,2018-04-06,NaT,CA3,...,7500.0,入会費無料,7.500000,7.0,10.0,6.0,1.0,2019-04-30,12.0,NaT
4,201903,AS460629,8,6.0,XXXXX,C01,M,2018-07-14,NaT,CA1,...,10500.0,通常,7.555556,8.0,9.0,6.0,1.0,2019-04-30,9.0,NaT


##予測する月の在籍期間を作成

In [15]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
  delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
  predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201809,OA909231,9,8.0,XX,C03,M,2018-01-01,NaT,CA1,...,7.083333,7.0,12.0,3.0,1.0,2019-04-30,15.0,NaT,8,2018-09-01
1,201805,HD862235,4,4.0,XXXXX,C01,M,2015-08-01,NaT,CA1,...,4.166667,4.0,6.0,1.0,1.0,2019-04-30,44.0,NaT,33,2018-05-01
2,201811,IK192713,5,6.0,XXXX,C01,M,2016-04-01,NaT,CA1,...,3.916667,3.5,6.0,2.0,1.0,2019-04-30,36.0,NaT,31,2018-11-01
3,201810,IK429268,6,7.0,XXXX,C02,F,2018-04-06,NaT,CA3,...,7.500000,7.0,10.0,6.0,1.0,2019-04-30,12.0,NaT,5,2018-10-01
4,201903,AS460629,8,6.0,XXXXX,C01,M,2018-07-14,NaT,CA1,...,7.555556,8.0,9.0,6.0,1.0,2019-04-30,9.0,NaT,7,2019-03-01


##欠損値を除去

In [16]:
predict_data.isna().sum()

,0
年月,0
customer_id,0
count_0,0
count_1,270
name,0
class,0
gender,0
start_date,0
end_date,2842
campaign_id,0


In [17]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

,0
年月,0
customer_id,0
count_0,0
count_1,0
name,0
class,0
gender,0
start_date,0
end_date,2624
campaign_id,0


##文字列型の変数を処理できるように整形

In [18]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,ナイト,M,8.0,1.0,8,0.0
1,通常,オールタイム,M,4.0,1.0,33,0.0
2,通常,オールタイム,M,6.0,1.0,31,0.0
3,入会費無料,デイタイム,F,7.0,1.0,5,0.0
4,通常,オールタイム,M,6.0,1.0,7,0.0


In [19]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,8.0,1.0,8,0.0,False,False,True,False,False,True,False,True
1,4.0,1.0,33,0.0,False,False,True,True,False,False,False,True
2,6.0,1.0,31,0.0,False,False,True,True,False,False,False,True
3,7.0,1.0,5,0.0,False,True,False,False,True,False,True,False
4,6.0,1.0,7,0.0,False,False,True,True,False,False,False,True


In [20]:
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,8.0,1.0,8,0.0,False,False,False,False,False
1,4.0,1.0,33,0.0,False,False,True,False,False
2,6.0,1.0,31,0.0,False,False,True,False,False
3,7.0,1.0,5,0.0,False,True,False,True,True
4,6.0,1.0,7,0.0,False,False,True,False,False


##決定木を用いて退会予測モデルを作成

In [22]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1.

In [23]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1813,0.0,0.0
1498,0.0,0.0
1657,0.0,0.0
763,1.0,1.0
1468,0.0,0.0


##予測モデルの評価とモデルのチューニング

In [24]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8954372623574145


In [25]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8954372623574145
0.973384030418251


In [26]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9315589353612167
0.9277566539923955


##モデルに寄与している変数の確認

In [27]:
importance = pd.DataFrame({"featuer_names":X.columns, "coefficient":model.feature_importances_})
importance

,featuer_names,coefficient
0,count_1,0.336747
1,routine_flg,0.119370
2,period,0.540718
3,campaign_name_入会費半額,0.002823
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000342
6,class_name_デイタイム,0.000000
7,gender_F,0.000000


##顧客の退会を予測

In [28]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [29]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [30]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.0326087 0.9673913]]
